## 搭建一个简单的前馈神经网络
### 典型的训练过程
1. 定义一个包含可学习参数的神经网络
2. 在数据集上迭代
3. 通过神经网络处理输入
4. 计算损失
5. 将梯度反向传播回神经网络的参数
6. 更新参数，更新算法：$w = w + \eta *grad$

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 分别代表数据的维度、卷积通道数、卷积核大小
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # 线性：y = wx + b，前一个是输入特征数，后一个是输出特征数
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    # 向前传播
    def forward(self, x):
        # 池化卷积层，并使用修正线性单元激活函数
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # 重新给定x的形状
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [36]:
net = Net()
print(net)

# 查看可学习参数
params = list(net.parameters())
print(len(params))
# print(params)
print([p.size() for p in params])    # 查看权重

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
[torch.Size([6, 1, 3, 3]), torch.Size([6]), torch.Size([16, 6, 3, 3]), torch.Size([16]), torch.Size([120, 576]), torch.Size([120]), torch.Size([84, 120]), torch.Size([84]), torch.Size([10, 84]), torch.Size([10])]


In [38]:
# 输入一个样本,查看输出结果
input = torch.randn(1, 1, 32, 32)
# print(input)
output = net(input)
print(output)

tensor([[-1.3347e-01, -4.9062e-02,  5.9137e-02,  1.4116e-04,  1.2130e-01,
          1.3707e-02, -3.1772e-02,  1.3489e-02, -1.5357e-01, -2.5954e-02]],
       grad_fn=<AddmmBackward>)


In [39]:
net.zero_grad()       # 初始梯度设置为0，并开始向后传播
output.backward(torch.randn(1, 10))
print(output)

tensor([[-1.3347e-01, -4.9062e-02,  5.9137e-02,  1.4116e-04,  1.2130e-01,
          1.3707e-02, -3.1772e-02,  1.3489e-02, -1.5357e-01, -2.5954e-02]],
       grad_fn=<AddmmBackward>)


In [47]:
# 使用损失函数
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)     # 改变样本形状与output一致
print(target)
# print('优化前', output)
# 使用mse损失函数
ceiterion = nn.MSELoss()
loss = ceiterion(output, target)
# print('优化后', output)
print('预测值与实际值的差异', loss)

tensor([[ 1.7717, -1.1456, -0.8468,  0.5295,  0.0641, -1.0548,  1.5503, -0.1480,
         -1.3860, -0.2634]])
预测值与实际值的差异 tensor(1.1182, grad_fn=<MseLossBackward>)


In [52]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [53]:
# 将所有参数的梯度的缓冲区归0
net.zero_grad()
print('反向传播前的梯度', net.conv1.bias.grad)

loss.backward()
print('反向传播后的梯度', net.conv1.bias.grad)

反向传播前的梯度 tensor([0., 0., 0., 0., 0., 0.])
反向传播后的梯度 tensor([-0.0081,  0.0200,  0.0494,  0.0368, -0.0217, -0.0381])


In [57]:
# 梯度下降算法
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [55]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

In [56]:
loss

tensor(1.0739, grad_fn=<MseLossBackward>)